<a href="https://colab.research.google.com/github/hellojustxn/nlp-projects/blob/main/NLP_Classifier_Sequential_Model_Attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip uninstall -y tensorflow tf-nightly keras

# !pip install -q -U tf-nightly
# !pip install -q -U tensorflow-text-nightly
!pip install tensorflow-text==2.6.0

     |████████████████████████████████| 4.4 MB 9.0 MB/s 


In [2]:
import pandas as pd
import numpy as np
import collections
import pathlib
import re
import string

import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

tf.executing_eagerly()

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)
import pathlib

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

2.6.0


In [3]:
%%script echo skipping
abalone_train = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv",
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
          "Viscera weight", "Shell weight", "Age"])

abalone_train.head()

skipping


In [4]:
%%script echo skipping
abalone_features = abalone_train.copy()
abalone_labels = abalone_features.pop('Age')
abalone_labels.head()


skipping


In [5]:
%%script echo skipping
abalone_model = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)                     
])

abalone_model.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam())
abalone_model.fit(abalone_features, abalone_labels, epochs=100)

abalone_model.summary()

skipping


In [6]:
%%script echo skipping
# abalone_model(abalone_features)
abalone_features.to_numpy()
abalone_np = abalone_features.to_numpy()
print(abalone_np[:1,].shape)
abalone_labels.head()
abalone_model.predict(abalone_np[:80,])

skipping


In [7]:
TRAINING_URL = 'https://storage.googleapis.com/timely-train/train.txt'
LABEL_URL = 'https://storage.googleapis.com/timely-train/label.txt'

utils.get_file('train.txt', origin=TRAINING_URL)
text_dir = utils.get_file('label.txt', origin=LABEL_URL)

parent_dir = pathlib.Path(text_dir).parent
list(parent_dir.iterdir())

188416/178972 [===============================] - 0s 0us/step


[PosixPath('/root/.keras/datasets/train.txt'),
 PosixPath('/root/.keras/datasets/label.txt')]

# Load the dataset

In [8]:
# Combine the dataset and the labels
lines_dataset = tf.data.TextLineDataset(str(parent_dir/'train.txt'))
lines_labeled_dataset = tf.data.TextLineDataset(str(parent_dir/'label.txt'))

# Sanity check
# for i in lines_dataset:
#   print(i)
# Output: tf.Tensor(b'EART - Monday, Wednesday, Thursday, Friday, Sunday 07:46 - 08:43', shape=(), dtype=string)

# print(type(lines_dataset))
# Output: <class 'tensorflow.python.data.ops.readers.TextLineDatasetV2'>

# Create a list of tuples containing (train, label) into a dataset
# Label the data




# all_labeled_data_set = tf.data.Dataset.from_tensor_slices(tensor1)
# labeled_data_set = tf.data.Dataset.from_tensor_slices(list(map(lambda x, y: (x, y), tf.constant(lines_dataset, dtype=string), lines_labeled_dataset)))

# Good since
# Output: (<tf.Tensor: shape=(), dtype=string, numpy=b'BIOE - Monday, Tuesday, Wednesday, Friday, Saturday, Sunday from 12:12 to 12:57'>, <tf.Tensor: shape=(), dtype=string, numpy=b'[[1, 1, 1, 0, 1, 1, 1], 0.51, 0.54]'>)
# But this is of type map, instead we want type dataset.
# labeled_data_set = list(map(lambda x, y: (x, y), lines_dataset, lines_labeled_dataset))
# print(labeled_data_set)
lines_list = list(map(lambda x: x, lines_dataset))
labels_list = list(map(lambda x: x, lines_labeled_dataset))

# print(type(labeled_data_set))
# Output: <class 'tensorflow.python.data.ops.readers.TextLineDatasetV2'>

# Zip the lines and labels together
# Create a list of labeled data
# labeled_data_list = [(i,j) for i, j in zip(lines_list, lables_list)]
# print(lines_list)
labeled_dataset = tf.data.Dataset.from_tensor_slices((lines_list, labels_list))

# for count,i in enumerate(lines_data_set.as_numpy_iterator()):
#   print(count)
# print(lines_data_set.cardinality())
# for element in lines_data_set:
#   print(element)

# Zip the two data sets
# labeled_data_set = tf.data.Dataset.zip((lines_data_set, labels_data_set))
# print(type(labeled_data_set))
# for i in labeled_data_set:
#   print(i)
#   print(i[0])

# Output: (<tf.Tensor: shape=(), dtype=string, numpy=b'BIOE - Monday, Tuesday, Wednesday, Friday, Saturday, Sunday from 12:12 to 12:57'>, <tf.Tensor: shape=(), dtype=string, numpy=b'[[1, 1, 1, 0, 1, 1, 1], 0.51, 0.54]'>)

Shuffle the data

In [9]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
VALIDATION_SIZE = 1000

In [10]:
labeled_dataset = labeled_dataset.shuffle(
  BUFFER_SIZE, reshuffle_each_iteration=False
)

# print(type(labeled_dataset))

note: printf and print have different behaviours on a tensor

In [11]:
for text, label in labeled_dataset.take(1):
  print("Sentence: ", text)
  print("Label: ", label)

Sentence:  tf.Tensor(b'ESCI: Monday, Saturday, Sunday 00:08 - 00:41', shape=(), dtype=string)
Label:  tf.Tensor(b'[[1, 0, 0, 0, 0, 1, 1], 0.01, 0.03]', shape=(), dtype=string)


# Prepare the dataset for training
To preprocess our text dataset, I use the [tf.text API](https://www.tensorflow.org/text) to tokenize, build a vocabulary and create a mapping between tokens to integers (StaticVocabularyTable) for the model

In [12]:
tokenizer = tf_text.UnicodeScriptTokenizer()

This function will lowercase and tokenize the text data set with the UnicodeScriptTokenizer. 

In [13]:
# Input from map as (Text, Label)
def tokenize(text, unused_label):
  lower_case = tf_text.case_fold_utf8(text)
  return tokenizer.tokenize(lower_case)

In [14]:
tokenized_dataset = labeled_dataset.map(tokenize)

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


In [15]:
# for text_batch in tokenized_dataset.take(5):
#   print(text_batch)
print(tokenized_dataset.cardinality())

tf.Tensor(5000, shape=(), dtype=int64)


Build a vocabulary and sort the tokens by frequency, keep the top VOCAB_SIZE tokens

In [16]:
VOCAB_SIZE = 10000

# AUTOTUNE = tf.data.AUTOTUNE class for configure_dataset used for performance
# TODO: Find out what config_ds does and how it affects the dataset
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)
tokenized_dataset = configure_dataset(tokenized_dataset)
# help(configure_dataset)




Create the vocabulary frequency dictionary

In [17]:
vocab_dict = collections.defaultdict(lambda: 0)

for tokens in tokenized_dataset.as_numpy_iterator():
  for tok in tokens:
    vocab_dict[tok] +=1 

# Sort by frequency, an example item is a tuple of the form (token, frequency)
# Ascending order
vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)

# Get only the tokens
vocab = [token for token, count in vocab]

# Get the VOCAB_SIZE highest frequency tokens
vocab = vocab[:VOCAB_SIZE]

# Get the length of the vocab, account for the case where len(vocab) < VOCAB_SIZE
vocab_size = len(vocab) 

print("First five vocab entries:", vocab[:10])

First five vocab entries: [b',', b'monday', b'tuesday', b'saturday', b'sunday', b'thursday', b'friday', b'wednesday', b'to', b'from']


Use the vocab to create a StaticVocabularyTable, map the tokens to integers in the range (2, vocab_size + 2). 

For the ```TextVectorization``` layer,

0 is reserved and will denote padding
1 is reserved and will denote OOV (Out of vocabulary) token

In

In [18]:
keys = vocab
values = range(2, len(vocab) + 2) # range(inclusive, exclusive) TODO: Revisit

# Data used to initialize a StaticVocabularyTable
init = tf.lookup.KeyValueTensorInitializer(
    keys, values, key_dtype=tf.string, value_dtype=tf.int64
)

# Number of buckets to use for OOV keys
num_oov_buckets = 1

# String to Id table that assigns out-of-vocabulary keys to hash buckets.
# Create a table to map keys-values tensors.
# key: tokens to values: integers
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)

In [19]:
# standardize
# tokenize
# vectorize
def preprocess_text(text, label):
  standardized = tf_text.case_fold_utf8(text)
  tokenized = tokenizer.tokenize(standardized)
  vectorized = vocab_table.lookup(tokenized)
  return vectorized, label

sample, label = preprocess_text("test hello", 1)
print(sample)

tf.Tensor([3902 3902], shape=(2,), dtype=int64)


In [20]:
example_text, example_label = next(iter(labeled_dataset))

print("Sentence: ", example_text.numpy())
vectorized_text, example_label = preprocess_text(example_text, example_label)
print("Vectorized: ",)


Sentence:  b'ESCI: Monday, Saturday, Sunday 00:08 - 00:41'
Vectorized: 


Encoding all label data

In [21]:
all_encoded_data = labeled_dataset.map(preprocess_text)



# Split the dataset into train and test

"For perfect shuffling, a buffer size greater than or equal to the full size of the dataset is required." Tf docs

In [22]:
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE)
print(train_data)
validation_data = all_encoded_data.take(VALIDATION_SIZE)

<ShuffleDataset shapes: ((None,), ()), types: (tf.int64, tf.string)>


TODO: Use the Keras ```TextVectorization``` layer to batch and pad the vectorized data.


Batches need to be of the same size and shape (each line of text has a different number of words)

In [23]:
# padded_batch: Combines consecutive elements of this dataset into padded batches.
train_data = train_data.padded_batch(BATCH_SIZE)
validation_data = validation_data.padded_batch(BATCH_SIZE)

sample_text, sample_labels = next(iter(validation_data))
print("Text batch shape: ", sample_text.shape)
print("Label batch shape: ", sample_labels.shape)
# print("First text example: ", sample_text)
# print("First label example: ", sample_labels)

Text batch shape:  (64, 23)
Label batch shape:  (64,)


In [24]:
vocab_size += 2 # To account for padding denoted with a 0, and OOV denoted with a 1

In [25]:
train_data = configure_dataset(train_data)
validation_data = configure_dataset(validation_data)

# Train the model

 ```int``` vectorized layer to build a 1D CN

In [26]:
NUM_LABELS = 1

def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [29]:
model = create_model(vocab_size=vocab_size, num_labels=3)
model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

# Issue #1: The labels are breaking the code
# Labels are of type string
# Issue #2: Sequential models cannot produce multiple outputs
history = model.fit(train_data, validation_data=validation_data, epochs=3)


Epoch 1/3


UnimplementedError: ignored